In [ ]:
%load_ext autoreload
%autoreload 2

%cd '..'

In [ ]:
import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s [%(levelname)s] %(message)s",
#     handlers=[
#         logging.FileHandler("data/logs/preprocess_comments_notebook.log"),
#         logging.StreamHandler()
#     ]
# )

In [ ]:
from tqdm import tqdm
tqdm.pandas()

from load.utils import (
    load_df_from_parquet,
)
from preprocessing.utils import (
    tokenize_comment,
    save_event_comments,
    build_vocab,
    save_event_vocab,
)

from preprocessing.constants import (
    ELECTIONS_EVENTS_INFO,
    MIN_OCCURENCE_FOR_VOCAB,
)

In [ ]:
YEAR = 2008
START_MONTH = 1
STOP_MONTH = 12

EVENT_NAME = f"us_elections_{YEAR}"
EVENT_INFO = ELECTIONS_EVENTS_INFO[EVENT_NAME]

## Filter event comments

In [ ]:
logging.info("Load partisan comments")

partisan_comments = load_df_from_parquet(
    file_name=f"partisan_comments_{YEAR}.parquet",
)


In [ ]:
logging.info("Filtering event data based on keywords...")
event_comments = partisan_comments[
    partisan_comments["body_cleaned"].str.contains(
        EVENT_INFO["regex"],
        regex=True,
    )
].copy()
logging.info("finished keyword filtering")

In [ ]:
logging.info(event_comments.shape)

In [ ]:
logging.info("save temp event comments")
save_event_comments(event_comments, f"temp_{EVENT_NAME}")


In [ ]:
del partisan_comments

In [ ]:
logging.info(event_comments.dtypes)

## Tokenize and stem comments

In [ ]:
logging.info("Tokenizing comments...")
event_comments["tokens"] = event_comments["body_cleaned"].progress_apply(
    tokenize_comment,
).astype("string")
logging.info("Finish tokenizing comments")

In [ ]:
logging.info(event_comments.dtypes)

In [ ]:
logging.info(f"Nr of event comments: {len(event_comments)}")

In [ ]:
logging.info("saving event comments...")
save_event_comments(event_comments, EVENT_NAME)

## Build event vocabulary

In [ ]:
# Read event data
# events_comments = load_event_comments(
#     event_comments,
#     EVENT_NAME,
#     file_type="parquet",
# )

event_vocab = build_vocab(
    event_comments["tokens"],
    min_comment_freq=MIN_OCCURENCE_FOR_VOCAB,
)


In [ ]:
logging.info("Vocabulary length")
logging.info(len(event_vocab))

In [ ]:
logging.info("Saving event vocab")
save_event_vocab(event_vocab, EVENT_NAME)